К сожалению, у меня не хватит времени прикрутить сюда распределенную обработку, поэтому в работе использую только часть данных.

In [75]:
import nltk
import re
import pymorphy2 as pm2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from gensim.models import Word2Vec, FastText
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline
from joblib import parallel_backend, Parallel, delayed

from tqdm import tqdm
from ast import literal_eval
from sklearn.metrics import pairwise_distances, classification_report
from sklearn.model_selection import train_test_split
from annoy import AnnoyIndex

import torch
from sklearn.mixture import BayesianGaussianMixture, GaussianMixture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from lightgbm import LGBMClassifier

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/avagadro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/avagadro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

__load data__

In [76]:
news = pd.read_csv('data/lenta-ru-news.csv', dtype={'topic': str})
news.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [9]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800975 entries, 0 to 800974
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     800975 non-null  object
 1   title   800975 non-null  object
 2   text    800970 non-null  object
 3   topic   738973 non-null  object
 4   tags    773756 non-null  object
 5   date    800975 non-null  object
dtypes: object(6)
memory usage: 36.7+ MB


__preprocessing__

Обрабатывать обучающие данные проще пайплайном, но для единичных пользовательских запросов пайплайн в реализованном виде не очень удобен,
т.к. требует конвертации в pd.DataFrame.

Оптимальное решение пока не нашел, и чтобы не писать тут одни и те же предобработки, пользуюсь конвертацией единичного запроса. В проде такое, на мой взгляд, недопустимо.

In [77]:
class BasicTransformer(TransformerMixin):
    def __init__(self, fields, **kwargs):
        self.fields = fields if isinstance(fields, list) else [fields]
        self.backend = kwargs.pop('backend', 'loky')
        self.n_jobs = kwargs.pop('n_jobs', -1)
        for key, value in kwargs.items():
            setattr(self, key, value)

    def fit(self, X, y=None, **fit_params):
        if 'all' in self.fields:
            self.fields = X.columns
        self.fields = [col for col in self.fields if col in X.columns]
        return self

    def transform(self, X, y=None):
        X = X.copy()
        for f in self.fields:
            X[f] = self.transform_column(X[f])
        return X
    
    def transform_column(self, column):
        return column

In [78]:
class Lowercase(BasicTransformer):
    def transform_column(self, column):
        print(f'Lowercase `{column.name}`')
        return column.str.lower()

class CutRegexp(BasicTransformer):
    def transform_column(self, column):
        print(f'Cut regexp `{column.name}`')
        return column.str.replace(self.pattern, self.fill, regex=True)

class Tokenize(BasicTransformer):
    def transform_column(self, column):
        with parallel_backend(self.backend, n_jobs=self.n_jobs):
            result = Parallel()(delayed(self.tokenizer)(row) for row in tqdm(column.values, desc=f'tokenize `{column.name}`'))
        return result

class ClearStopwords(BasicTransformer):
    def transform_column(self, column):
        def _drop_stopwords(tokens):
            # return [w for w in tokens if w not in self.stopwords]     # NOTE проверить, какой вариант быстрее
            return list(set(tokens).difference(self.stopwords))

        with parallel_backend(self.backend, n_jobs=self.n_jobs):
            result = Parallel()(delayed(_drop_stopwords)(row) for row in tqdm(column.values, desc=f'clearing `{column.name}`'))
        return result
        # return column.apply(lambda tokens: [w for w in tokens if w not in self.stopwords])

class Lemmatize(BasicTransformer):
    morph = pm2.MorphAnalyzer()
    def transform_column(self, column):
        def _lemma(tokens):
            return [self.morph.parse(w)[0].normal_form for w in tokens]

        with parallel_backend(self.backend, n_jobs=self.n_jobs):
            result = Parallel()(delayed(_lemma)(row) for row in tqdm(column.values, desc=f'lemmatize `{column.name}`'))
        return result

In [79]:
# init pipeline
fields = ['title', 'text']
pipeline = make_pipeline(
    Lowercase(fields),
    CutRegexp(fields, pattern=r'\W|\d', fill=' '),      # убрать спецсимволы и цифры
    Tokenize(fields, tokenizer=nltk.tokenize.word_tokenize),
    ClearStopwords(fields, stopwords=nltk.corpus.stopwords.words('russian')),
    Lemmatize(fields)
)

In [5]:
# prepare masks
nans = news['text'].isna() | news['title'].isna()       # drop nan
short = news['text'].str.split().str.len() < 10         # drop short texts - это очень тяжелая операция

topics_mask = news['topic'].value_counts() > 5000       # select data with most frequent topics - это, опять же, для упрощения задачи
topics = news['topic'].value_counts()[topics_mask].index
use_topics = news['topic'].isin(topics)

In [149]:
# preprocess data sample: поскольку времени мало, беру очень маленькую часть данных
data = news[~nans & ~short & use_topics].sample(25000, random_state=23)

prepared = pipeline.fit_transform(data)
prepared.head()

Lowercase `title`
Lowercase `text`
Cut regexp `title`
Cut regexp `text`


lemmatize `text`: 100%|██████████| 25000/25000 [07:29<00:00, 55.56it/s]


,url,title,text,topic,tags,date
69783,https://lenta.ru/news/2003/04/23/teacher/,"[хорватский, библиотека, домашний, учитель, ед...","[сосед, госпитализация, сообщать, ananova, пре...",Из жизни,Все,2003/04/23
69373,https://lenta.ru/news/2003/04/15/syria/,"[буш, готовить, запретить, война, пентагон, си...","[вашингтон, эпизод, принятие, новый, однако, г...",Мир,Все,2003/04/15
722211,https://lenta.ru/news/2018/08/02/latte_for_pre...,"[беременный, средство, вместо, кофе, канадка, ...","[чистить, развесить, заявить, однако, франчайз...",Из жизни,Люди,2018/08/02
240806,https://lenta.ru/news/2008/08/08/mechel1/,"[акция, перенести, размещение, мечел, второй]","[потерять, акция, наметить, доллар, металлурги...",Экономика,Все,2008/08/08
190561,https://lenta.ru/news/2007/05/29/nba/,"[вылет, оказаться, плей, андрей, ют, кириленко...","[сперс, забить, сборная, официальный, чужой, к...",Спорт,Все,2007/05/29


In [151]:
prepared.to_csv('data/prepared.csv')

In [80]:
# load saved
prepared = pd.read_csv('data/prepared.csv', index_col='Unnamed: 0', converters={'title': literal_eval, 'text': literal_eval})

In [110]:
field = 'title'
# field = 'text'
sentences = prepared[field]

# request preprocess
request_text = 'спортивные достижения'
request = pipeline.fit_transform(pd.DataFrame([request_text], columns=[field]))
request

Lowercase `title`
Cut regexp `title`


lemmatize `title`: 100%|██████████| 1/1 [00:00<00:00, 1190.89it/s]


,title
0,"[достижение, спортивный]"


__word2vec__

In [167]:
# word2vec init
vecsize = 300
w2v = Word2Vec(sentences, vector_size=vecsize, window=5, min_count=1, seed=31)
# build embeddings
w2v_embs = sentences.apply(lambda row: np.mean([w2v.wv[word] for word in row if word in w2v.wv], axis=0))

# calc request embedding
w2v_request_emb = request[field].apply(lambda row: np.array([w2v.wv[word] for word in row if word in w2v.wv]).mean(axis=0))

# build w2v cluster map
w2v_cluster = AnnoyIndex(vecsize ,'angular')
for idx, item in enumerate(w2v_embs):
    w2v_cluster.add_item(idx, item)

w2v_cluster.build(10, n_jobs=-1)

# get top nearest
top = w2v_cluster.get_nns_by_vector(w2v_request_emb[0], 10)
news.loc[top]

,url,title,text,topic,tags,date
20774,https://lenta.ru/news/2001/01/29/diplomat/,"Канада требует выдачи российских дипломатов, у...",Канада обратилась к России с просьбой лишить д...,Мир,Все,2001/01/29
18277,https://lenta.ru/news/2000/12/14/opros/,Доверие к доллару в России падает,Число россиян - сторонников доллара и другой и...,Экономика,Все,2000/12/14
2933,https://lenta.ru/news/1999/12/27/chaos/,В Берлине открывается конгресс Chaos Computer ...,В понедельник в Берлине открывается шестнадцат...,Интернет и СМИ,Все,1999/12/27
12217,https://lenta.ru/news/2000/08/31/robot/,Роботы научились ползать и размножаться,Ученые из Brandeis University в Массачусеттсе ...,Интернет и СМИ,Все,2000/08/31
23574,https://lenta.ru/news/2001/03/19/insects/,Лондонские гурманы перешли на насекомых в шоко...,Среди лондонцев все популярнее делается новое ...,Из жизни,Все,2001/03/19
18781,https://lenta.ru/news/2000/12/22/ethics/,Депутат избил водителя. Теперь его будут судит...,Около четырехсот жалоб на поведение депутатов ...,Россия,Все,2000/12/22
13413,https://lenta.ru/news/2000/09/23/shooting/,Преступники расстреляли жертву на оживленной у...,В субботу около 15 часов у дома 27 по улице Кр...,Россия,Все,2000/09/23
1387,https://lenta.ru/news/1999/11/02/wood/,Китайскую внешнюю торговлю разъедают жуки-короеды,"Миллиарды долларов инвестиций, вложенные в тор...",Экономика,Все,1999/11/02
19342,https://lenta.ru/news/2001/01/04/cz/,Чешский сенат стал на защиту тележурналистов,Верхняя палата парламента Чехии в среду вечеро...,Мир,Все,2001/01/04
20805,https://lenta.ru/news/2001/01/30/dagestan/,В депутата парламента Дагестана бросили гранату,В дагестанском городе Избербаше совершено поку...,Россия,Все,2001/01/30


In [168]:
# search nearest with simple distancing
distances = pairwise_distances(np.array(list(w2v_request_emb.values)), np.array(list(w2v_embs.values)), metric='cosine').flatten()
distances = pd.Series(distances, index=prepared.index)      # restore index

sorted_distances = distances.sort_values(ascending=False)
# top 10 news
top = sorted_distances[:10].index
# overview
news.loc[top]

,url,title,text,topic,tags,date
610286,https://lenta.ru/news/2016/08/02/mamont/,Мамонтов добила жажда,Последняя популяция мамонтов на Земле вымерла ...,Наука и техника,Наука,2016/08/02
711456,https://lenta.ru/news/2018/05/03/poslemayskih/,Орангутан загрустил в неволе и растолстел,Орангутан из Бангкокского зоопарка в Таиланде ...,Из жизни,Звери,2018/05/03
733934,https://lenta.ru/news/2018/11/06/ronaldinho/,Роналдиньо подозрительно обеднел,Бывший полузащитник «Барселоны» и сборной Браз...,Спорт,Футбол,2018/11/06
452612,https://lenta.ru/news/2013/01/30/tezzz/,Tequilajazzz воссоединится для перезаписи «Цел...,Группа Tequilajazzz перезапишет альбом «Целлул...,Культура,Музыка,2013/01/30
180927,https://lenta.ru/news/2007/02/21/congress/,В Таллин съезжаются каббалисты,22 января в Таллине откроется Европейский конг...,Бывший СССР,Все,2007/02/21
491914,https://lenta.ru/news/2013/12/24/fbreader/,YotaPhone подружили с читалкой FBReader,Приложение для чтения электронных книг FBReade...,Наука и техника,Гаджеты,2013/12/24
112313,https://lenta.ru/economy/2005/02/11/yugansk/,"""Юганскнефтегаз"" рассчитался с Минприроды","""Юганскнефтегаз"" уже направил в Министерство п...",Экономика,Все,2005/02/11
107,https://lenta.ru/news/1999/09/07/inkombank/,"""Инкомбанк"" отстоял свою состоятельность","""Инкомбанк"" добился приостановления дела о его...",Россия,Все,1999/09/07
288524,https://lenta.ru/news/2009/06/26/needlework/,Сикстинскую капеллу вышили крестиком,"Джоанна Лопяновски-Робертс, живущая в США урож...",Культура,Все,2009/06/26
478790,https://lenta.ru/news/2013/09/06/guelman/,Гельмана выселили с «Винзавода»,Галерею «Культурный Альянс. Проект Марата Гель...,Культура,Искусство,2013/09/06


__FastText__

In [172]:
# FastText init
vecsize = 300
ft = FastText(sentences, vector_size=vecsize, min_count=20, seed=31)
# build embeddings
ft_embs = sentences.apply(lambda row: np.mean([ft.wv[word] for word in row if word in ft.wv], axis=0))

# calc request embedding
ft_request_emb = request[field].apply(lambda row: np.array([ft.wv[word] for word in row if word in ft.wv]).mean(axis=0))

# build w2v cluster map
ft_cluster = AnnoyIndex(vecsize ,'angular')
for idx, item in enumerate(w2v_embs):
    ft_cluster.add_item(idx, item)

ft_cluster.build(10, n_jobs=-1)

# get top nearest
top = ft_cluster.get_nns_by_vector(ft_request_emb[0], 10)
news.loc[top]
# NOTE можно дополнительно сделать отбор или ранжирование кандидатов по топкику

,url,title,text,topic,tags,date
14807,https://lenta.ru/news/2000/10/17/visas/,Чехия переходит на визовый режим со всеми стра...,C 22 октября Чехия вводит визовый пограничный ...,Мир,Все,2000/10/17
1672,https://lenta.ru/news/1999/11/15/mazeikiu/,Остановлен единственный нефтезавод в странах Б...,Производство на нефтеперерабатывающем заводе M...,Экономика,Все,1999/11/15
16663,https://lenta.ru/news/2000/11/18/bitum/,Пожар на битумном заводе тушили больше часа,Сильный пожар произошел во второй половине дня...,Россия,Все,2000/11/18
14985,https://lenta.ru/news/2000/10/20/potter_frodo/,Гарри Поттер померяется силами с Фродо и Гэнда...,"Эдриан Бюрн, руководитель отдела продаж издате...",Культура,Все,2000/10/20
22120,https://lenta.ru/news/2001/02/20/angel/,"Больничная сиделка призналась, что убивала сво...","23-летняя больничная сиделка, прозванная ""черн...",Мир,Все,2001/02/20
21681,https://lenta.ru/news/2001/02/12/protest/,Полиция арестовала 325 шотландцев за борьбу с ...,В понедельник толпа шотландцев блокировала вхо...,Мир,Все,2001/02/12
8705,https://lenta.ru/news/2000/06/08/baraev/,"Манилов знает, где искать Басаева","По данным военных, чеченский полевой командир ...",Россия,Все,2000/06/08
23991,https://lenta.ru/news/2001/03/26/investment/,Об иностранных инвесторах в Белоруссии позабот...,Белорусские службы безопасности решили гаранти...,Мир,Все,2001/03/26
23518,https://lenta.ru/news/2001/03/18/gantamirov/,В Ингушетии задержан похититель людей,В субботу вечером в Ингушетии задержан житель ...,Россия,Все,2001/03/18
2852,https://lenta.ru/news/1999/12/23/cherkesia/,В Карачаево-Черкесии подтасовали результаты вы...,В четверг прокуратурой Карачаево-Черкесии было...,Россия,Все,1999/12/23


In [173]:
# search nearest with simple distancing
distances = pairwise_distances(np.array(list(ft_request_emb.values)), np.array(list(ft_embs.values)), metric='cosine').flatten()
distances = pd.Series(distances, index=prepared.index)      # restore index

sorted_distances = distances.sort_values(ascending=False)
# top 10 news
top = sorted_distances[:10].index
# overview
news.loc[top]

,url,title,text,topic,tags,date
248000,https://lenta.ru/news/2008/09/29/digg/,Digg оценили в 175 миллионов долларов,В ходе последнего раунда финансирования социал...,Интернет и СМИ,Все,2008/09/29
114701,https://lenta.ru/news/2005/03/11/oil/,Кудрин не дал Фрадкову снизить НДС,Налог на добавленную стоимость в ближайшие три...,Экономика,Все,2005/03/11
470185,https://lenta.ru/news/2013/06/24/quest/,Dragon Quest X выпустят на PC,Square Enix выпустит версию MMORPG Dragon Ques...,Наука и техника,Игры,2013/06/24
16222,https://lenta.ru/news/2000/11/10/british/,British Telecom поделят и продадут,Ведущая телекоммуникационная компания Великобр...,Экономика,Все,2000/11/10
446367,https://lenta.ru/news/2012/12/08/yakzags/,Якутам запретили пить в ЗАГСе,Территорию управления ЗАГСа при правительстве ...,Россия,Все,2012/12/08
254845,https://lenta.ru/news/2008/11/13/supreme/,Square Enix выпустит Supreme Commander 2,Издательство Square Enix заключило партнерское...,Наука и техника,Все,2008/11/13
130166,https://lenta.ru/news/2005/09/13/sell/,"Ford Motor продает Hertz за 5,6 миллиарда долл...",Концерн Ford Motor продает свое подразделение ...,Экономика,Все,2005/09/13
455275,https://lenta.ru/news/2013/02/20/amnesia/,Сиквел Amnesia отложили,Инди-хоррор Amnesia: A Machine for Pigs отложи...,Наука и техника,Игры,2013/02/20
677832,https://lenta.ru/news/2017/08/16/tom/,Tom Ford дал маслу уда проявить себя в духах,Американская марка Tom Ford обновила парфюмерн...,Ценности,Внешний вид,2017/08/16
167613,https://lenta.ru/news/2006/10/09/symantec/,Symantec заработает на антивирусах 10 миллиард...,Главный исполнительный директор Symantec Джон ...,Интернет и СМИ,Все,2006/10/09


__why it doesn't work__

In [180]:
positive = request[field][0][1]
positive

'спортивный'

In [181]:
w2v.wv.most_similar(positive, topn=10)

[('адвокат', 0.9971103668212891),
 ('выпустить', 0.9970963597297668),
 ('бой', 0.9970950484275818),
 ('открыть', 0.9970911145210266),
 ('франция', 0.9970730543136597),
 ('сеть', 0.9970723390579224),
 ('отправить', 0.9970698356628418),
 ('газета', 0.9970691800117493),
 ('аэропорт', 0.9970670938491821),
 ('кубок', 0.9970661997795105)]

In [182]:
ft.wv.most_similar(positive, topn=10)

[('свободный', 0.9999927878379822),
 ('местный', 0.9999927878379822),
 ('навальный', 0.9999926686286926),
 ('неизвестный', 0.999991238117218),
 ('незаконный', 0.9999908208847046),
 ('рекламный', 0.9999904036521912),
 ('социальный', 0.999988853931427),
 ('красный', 0.9999886155128479),
 ('секретный', 0.9999885559082031),
 ('частный', 0.9999882578849792)]

Очень интересные семнатические расстояния между словами получаются.

__пример классификатора топика__

In [183]:
# select embeddings
# embeddings = w2v_embs
embeddings = ft_embs

embeddings = np.array(list(embeddings.values))
embeddings.shape

(25000, 300)

In [184]:
# кластеризатор по топикам
idx2topic = dict(enumerate(prepared['topic'].unique()))
topic2idx = {v: k for k,v in idx2topic.items()}

true_labels = prepared['topic'].map(topic2idx)

# train/valid split
embs = pd.DataFrame(embeddings, index=true_labels.index)    # restore embedding indices
train, valid = train_test_split(embs.index, test_size=0.2, stratify=true_labels, random_state=19)

In [191]:
# fit classifier
# model = BayesianGaussianMixture(n_components=len(topic2idx), n_init=1, random_state=17)
# model = KNeighborsClassifier(n_neighbors=len(topic2idx), n_jobs=-1)
# model = KMeans(n_clusters=len(topic2idx), random_state=17)
model = LGBMClassifier(n_estimators=500, learning_rate=0.1, max_depth=5, num_leaves=31, n_jobs=-1, random_state=17)
model.fit(embs.loc[train], true_labels[train])
pred_labels = model.predict(embs.loc[valid])
# print report
report = classification_report(true_labels[valid], pred_labels)
print(report)

              precision    recall  f1-score   support

           0       0.30      0.04      0.07       198
           1       0.34      0.44      0.38       905
           2       0.39      0.49      0.43       553
           3       0.66      0.54      0.59       431
           4       0.25      0.19      0.22       357
           5       0.00      0.00      0.00       139
           6       0.33      0.55      0.41      1086
           7       0.14      0.06      0.08       315
           8       0.18      0.16      0.17       361
           9       0.34      0.16      0.22       359
          10       0.00      0.00      0.00        53
          11       0.00      0.00      0.00        39
          12       0.50      0.19      0.28       151
          13       1.00      0.02      0.04        53

    accuracy                           0.35      5000
   macro avg       0.32      0.20      0.21      5000
weighted avg       0.34      0.35      0.32      5000



In [192]:
class EmbeddingsDataset():
    def __init__(self, X, y):
        self.X = X.reset_index(drop=True)
        self.y = y.reset_index(drop=True)

    def __getitem__(self, index):
        return self.X.loc[index].values, self.y.loc[index].values
    
    def __len__(self):
        return self.X.shape[0]


class Cell(torch.nn.Module):
    def __init__(self, inp, out, *, act=torch.relu, drop=0):
        super().__init__()
        self.linear = torch.nn.Linear(inp, out)
        self.activation = act
        self.bn = torch.nn.BatchNorm1d(out)
        self.dp = torch.nn.Dropout(drop) if drop else None
    
    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        x = self.bn(x)
        if self.dp is not None:
            x = self.dp(x)
        return x


class Net(torch.nn.Module):
    def __init__(self, inp, out):
        super().__init__()
        self.cell1 = Cell(inp, 512, act=torch.relu, drop=0.1)
        self.cell2 = Cell(512, 256, act=torch.relu, drop=0.1)
        self.cell3 = Cell(256, out, act=torch.relu, drop=0.1)
        
    def forward(self, x):
        x = self.cell1(x)
        x = self.cell2(x)
        x = self.cell3(x)
        return torch.softmax(x, dim=0)

In [193]:
EPOCHS = 5

# init network
device = 'cuda'
net = Net(vecsize, len(topic2idx)).to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

# make data loaders
onehot_labels = pd.DataFrame(np.eye(true_labels.max() + 1)[true_labels.values], index=true_labels.index)
emb_train_dataset = EmbeddingsDataset(embs.loc[train], onehot_labels.loc[train])
emb_valid_dataset = EmbeddingsDataset(embs.loc[valid], onehot_labels.loc[train])
train_loader = torch.utils.data.DataLoader(emb_train_dataset, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(emb_valid_dataset, batch_size=64, shuffle=True)

In [194]:
# train
net.train()
for ep in range(EPOCHS):
    sum_loss, items = 0.0, 0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Epoch {ep + 1}/{EPOCHS}')
    for i, batch in pbar:
        inputs, labels = batch[0].to(device).float(), batch[1].to(device).float()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        sum_loss += loss.item()
        items += len(labels)
        pbar.set_postfix({'cumulative loss per item': sum_loss / items})
print('\nDone.')

Epoch 5/5: 100%|██████████| 313/313 [00:04<00:00, 71.77it/s, cumulative loss per item=0.00463]


Done.


In [195]:
# evaluation
net.eval()
pred_labels = net(torch.as_tensor(embs.loc[valid].values, device=device)).detach().cpu().argmax(axis=1)
report = classification_report(true_labels[valid], pred_labels)
print(report)

/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.06      0.20      0.10       198
           1       0.27      0.06      0.09       905
           2       0.00      0.00      0.00       553
           3       0.94      0.24      0.39       431
           4       0.40      0.06      0.11       357
           5       0.00      0.00      0.00       139
           6       0.32      0.42      0.36      1086
           7       0.01      0.00      0.01       315
           8       0.14      0.21      0.17       361
           9       0.02      0.01      0.01       359
          10       0.03      0.47      0.06        53
          11       0.00      0.00      0.00        39
          12       0.08      0.50      0.14       151
          13       0.00      0.00      0.00        53

    accuracy                           0.17      5000
   macro avg       0.16      0.16      0.10      5000
weighted avg       0.25      0.17      0.16      5000



/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [196]:
#